In [ ]:
# default_exp pretrained

# Pretrained

> fast.ai ULMFiT helpers to easily use pretrained models

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import json
from fastai.text.all import SentencePieceTokenizer, SpacyTokenizer, language_model_learner, \
                            text_classifier_learner, untar_data, Path, patch, \
                            LMLearner, os, pickle, shutil, AWD_LSTM, accuracy, \
                            Perplexity, delegates

In [ ]:
#export
def _get_config(path):
    with open(path/'model.json', 'r') as f:
        config = json.load(f)
    return config

In [ ]:
#export
def _get_pretrained_model(url):
    fname = f"{url.split('/')[-1]}.tgz"
    path = untar_data(url, fname=fname, c_key='model')
    return path

In [ ]:
#export
def _get_direction(backwards):
    return 'bwd' if backwards else 'fwd'

In [ ]:
#export
def _get_class(classname):
    cls = None
    if len(classname.split('.')) > 1:
        comp = classname.rsplit('.', 1)
        imported = import_module(comp[0])
        cls = getattr(imported, comp[1])
    else:
        cls = globals()[classname]
    return cls

In [ ]:
#hide
assert(_get_direction(backwards=False) == 'fwd')
assert(_get_direction(backwards=True) == 'bwd')

Get `model` and `vocab` files from path. 

In [ ]:
#export
def _get_model_files(path, backwards=False, encoder=False):
    direction = _get_direction(backwards)
    #config = _get_config(path/direction)
    try: 
        model_path = path/direction
        if encoder:
            model_file = list(model_path.glob(f'*encoder.pth'))[0]
        else:
            model_file = list(model_path.glob(f'*model.pth'))[0]
            
        vocab_file = list(model_path.glob(f'*vocab.pkl'))[0]
        fnames = [model_file.absolute(),vocab_file.absolute()]
    except IndexError: print(f'The model in {model_path} is incomplete, download again'); raise
    fnames = [str(f.parent/f.stem) for f in fnames]
    return fnames

## Tokenizer

Get `tokenizer` from model-config. Tokenizer parameters in `model.json` will be passed to the Tokenizer. As of now SentencePiece and Spacy are supported.

In [ ]:
#export
def tokenizer_from_pretrained(url, pretrained=False, backwards=False, **kwargs):
    path = _get_pretrained_model(url)
    direction = _get_direction(backwards)
    config = _get_config(path/direction)
    
    if config['tokenizer']['class'] == 'SentencePieceTokenizer':
        if pretrained: config['tokenizer']['params']['sp_model'] = path/'spm'/'spm.model'
        
    tok_cls = _get_class(config['tokenizer']['class'])
    tok = tok_cls(**config['tokenizer']['params'])
    
    return tok

## Language Model Learner

Create `langauge_model_learner` from pretrained model-URL. All parameters will be passed to `language_model_learner`. The following parameters are set automatically: `arch`, `pretrained` and `pretrained_fnames`. By default `accuracy` and `perplexity` are passed as `metrics`. 

In [ ]:
#export
@delegates(language_model_learner)
def language_model_from_pretrained(dls, url=None, backwards=False, metrics=None, **kwargs):
    arch = AWD_LSTM # TODO: Read from config
    path = _get_pretrained_model(url)
    fnames = _get_model_files(path, backwards=backwards)
    metrics = [accuracy, Perplexity()] if metrics == None else metrics
    return language_model_learner(dls, 
                                  arch, 
                                  pretrained=True, 
                                  pretrained_fnames=fnames, 
                                  metrics=metrics,
                                  **kwargs)

In [ ]:
#export
def _get_model_path(learn=None, path=None):
    path = (learn.path/learn.model_dir) if not path else Path(path)
    if not path.exists(): os.makedirs(path, exist_ok=True)
    return path

Saves the following model files to `path`:
- Model (`lm_model.pth`)
- Encoder (`lm_encoder.pth`)
- Vocab from dataloaders (`lm_vocab.pkl`)
- SentencePieceModel (`spm/`)

In [ ]:
#export
@patch
def save_lm(x:LMLearner, path=None, with_encoder=True, backwards=False):
    path = _get_model_path(x, path)
    direction = _get_direction(backwards)
    model_path = path/direction
    if not model_path.exists(): os.makedirs(model_path, exist_ok=True)
        
    x.to_fp32()
    # save model
    x.save((model_path/'lm_model').absolute(), with_opt=False)
    
    # save encoder
    if with_encoder:
        x.save_encoder((model_path/'lm_encoder').absolute())

    # save vocab
    with open((model_path/'lm_vocab.pkl').absolute(), 'wb') as f:
        pickle.dump(x.dls.vocab, f)
       
    # save tokenizer if SentencePiece is used
    if isinstance(x.dls.tok, SentencePieceTokenizer):
        # copy SPM if path not spm path
        spm_path = Path(x.dls.tok.cache_dir)
        if path.absolute() != spm_path.absolute():
            target_path = path/'spm'
            if not target_path.exists(): os.makedirs(target_path, exist_ok=True)
            shutil.copyfile(spm_path/'spm.model', target_path/'spm.model')
            shutil.copyfile(spm_path/'spm.vocab', target_path/'spm.vocab')
    
    return path

## Text Classifier

In [ ]:
#def vocab_from_lm(learn=None, path=None):
#    path = _get_model_path(learn, path)
#    with open((path/'lm_vocab.pkl').absolute(), 'rb') as f:
#        return pickle.load(f)

In [ ]:
#def spm_from_lm(learn=None, path=None):
#    path = _get_model_path(learn, path)

Create `text_classifier_learner` from fine-tuned model path (saved with `learn.save_lm()`).

In [ ]:
#export
@delegates(text_classifier_learner)
def text_classifier_from_lm(dls, path=None, backwards=False, **kwargs):
    arch = AWD_LSTM # TODO: Read from config / _get_class()
    path = _get_model_path(path=path)
    fnames = _get_model_files(path, backwards=backwards, encoder=True)
    learn = text_classifier_learner(dls, arch, pretrained=False, **kwargs)
    learn.load_encoder(fnames[0])
    return learn

# Tests - Tokenizer, LM and Classifier

In [ ]:
#hide
#slow
url = 'http://localhost:8080/ulmfit-dewiki.tgz'
tok = tokenizer_from_pretrained(url, pretrained=True)
assert(tok.vocab_sz == 15000)
assert('ulmfit-dewiki/spm/spm.model' in str(tok.sp_model))

In [ ]:
#hide
#slow
tok = tokenizer_from_pretrained(url, pretrained=False)
assert(tok.sp_model == None)
assert(tok.vocab_sz == 15000)

In [ ]:
#hide
#slow
from fastai.text.all import AWD_LSTM, DataBlock, TextBlock, ColReader, RandomSplitter
import pandas as pd

backwards = False

df = pd.read_csv(Path('_test/data_lm_sample.csv'))

dblocks = DataBlock(blocks=(TextBlock.from_df('text', tok=tok, is_lm=True, backwards=backwards)),
                    get_x=ColReader('text'), 
                    splitter=RandomSplitter(valid_pct=0.1, seed=42))
dls = dblocks.dataloaders(df, bs=128)

learn = language_model_from_pretrained(dls, url=url, backwards=backwards)
learn.fit_one_cycle(1)

/home/florian/miniconda3/envs/fastai-ulmfit/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.461167,6.521255,0.170177,679.430847,00:00


In [ ]:
#hide
#slow
path = learn.save_lm()
vocab = learn.dls.vocab
path

Path('models')

In [ ]:
#hide
#slow
from fastai.text.all import AWD_LSTM, DataBlock, TextBlock, ColReader, RandomSplitter, CategoryBlock
import pandas as pd

backwards = False

df = pd.read_csv(Path('_test/data_class_sample.csv'))

dblocks = DataBlock(blocks=(TextBlock.from_df('text', tok=tok, vocab=vocab, backwards=backwards), CategoryBlock),
                    get_x=ColReader('text'), 
                    get_y=ColReader('label'))
dls = dblocks.dataloaders(df, bs=128)

learn = text_classifier_from_lm(dls, path=path, backwards=backwards)
learn.fit_one_cycle(1)
learn.get_preds()

/home/florian/miniconda3/envs/fastai-ulmfit/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


epoch,train_loss,valid_loss,time
0,0.744500,0.689982,00:02


(tensor([[0.5193, 0.4807],
         [0.4815, 0.5185],
         [0.5388, 0.4612],
         [0.5185, 0.4815],
         [0.5308, 0.4692],
         [0.5061, 0.4939],
         [0.5493, 0.4507],
         [0.5111, 0.4889],
         [0.5562, 0.4438],
         [0.5348, 0.4652],
         [0.5624, 0.4376],
         [0.4860, 0.5140],
         [0.4883, 0.5117],
         [0.4892, 0.5108],
         [0.4824, 0.5176],
         [0.5322, 0.4678],
         [0.5151, 0.4849],
         [0.4987, 0.5013],
         [0.4727, 0.5273],
         [0.5062, 0.4938],
         [0.4856, 0.5144],
         [0.5231, 0.4769],
         [0.4989, 0.5011],
         [0.5040, 0.4960],
         [0.4821, 0.5179],
         [0.5580, 0.4420],
         [0.5089, 0.4911],
         [0.5447, 0.4553],
         [0.5216, 0.4784],
         [0.4987, 0.5013],
         [0.5668, 0.4332],
         [0.4845, 0.5155],
         [0.4946, 0.5054],
         [0.4987, 0.5013],
         [0.4946, 0.5054],
         [0.5385, 0.4615],
         [0.5198, 0.4802],
 